In [1]:
import preprocessData
import calculate
from calculate import Math
from database import bondLengthDatabase, segmentVolumeDatabase
import numpy as np
import sys
import os
import csv
import SGmodel
import resultData

In [203]:
struct = []
f = []
molType = []
allEnergy = []
orgcom = []
spec = []
basises = []
for file in preprocessData.dataName():
    pdgod = preprocessData.getOutputData()
    if file[-3:] == 'out':
        sss = pdgod.dataMolecular(file)
        sss[0], sss[1] = sss[1], sss[0]
        temorg = pdgod.dataOrgcom(file)
        temEnergy = pdgod.dataEnergy(file)
        temStruct = pdgod.dataStructure(file)
        temBasis = pdgod.dataBasis(file)
        temType = pdgod.dataMoltype(file)
        if len(spec) == 0:
            spec.append(sss)
            orgcom.append(temorg)
            allEnergy.append(temEnergy)
            struct.append(temStruct)
            basises.append(temBasis)
            f.append(file)
            molType.append(temType)
        else:
            for i, spe in enumerate(spec):
                if sss[0][2]+sss[1][2] <= spe[0][2]+spe[1][2]:
                    spec.insert(i, sss)
                    orgcom.insert(i, temorg)
                    allEnergy.insert(i, temEnergy)
                    struct.insert(i, temStruct)
                    basises.insert(i, temBasis)
                    f.insert(i, file)
                    molType.insert(i, temType)
                    break       
                try:
                    if sss[0][2]+sss[1][2] > spe[0][2]+spe[1][2] and sss[0][2]+sss[1][2] <= spec[i+1][0][2]+spec[i+1][1][2]:
                        spec.insert(i+1, sss)
                        orgcom.insert(i+1, temorg)
                        allEnergy.insert(i+1, temEnergy)
                        struct.insert(i+1, temStruct)
                        basises.insert(i+1, temBasis)
                        f.insert(i+1, file)
                        molType.insert(i+1, temType)
                        break
                except:pass
                if i == len(spec)-1:
                    spec.append(sss)
                    orgcom.append(temorg)
                    allEnergy.append(temEnergy)
                    struct.append(temStruct)
                    basises.append(temBasis)
                    f.append(file)
                    molType.append(temType)
                    break
    else:continue
print(np.array(f))
print(np.array(orgcom))
print(np.array(spec))


['formaldehyde-formamide_jtz.out' 'formaldehyde-acetamide_jtz.out'
 'propanal-formamide_jtz.out' 'formaldehyde-propanamide_jtz.out'
 'acetaldehyde-acetamide_jtz.out' 'propanal-acetamide_jtz.out'
 'butanal-formamide_jtz.out' 'acetaldehyde-propanamide_jtz.out'
 'propanal-propanamide_jtz.out' 'butanal-acetamide_jtz.out'
 'butanal-propanamide_jtz.out']
[['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']
 ['Amide' 'Aldehyde']]
[[[1 0 1 2]
  [1 1 1 3]]

 [[1 0 1 2]
  [1 1 2 5]]

 [[1 0 3 6]
  [1 1 1 3]]

 [[1 0 1 2]
  [1 1 3 7]]

 [[1 0 2 4]
  [1 1 2 5]]

 [[1 0 3 6]
  [1 1 2 5]]

 [[1 0 4 8]
  [1 1 1 3]]

 [[1 0 2 4]
  [1 1 3 7]]

 [[1 0 3 6]
  [1 1 3 7]]

 [[1 0 4 8]
  [1 1 2 5]]

 [[1 0 4 8]
  [1 1 3 7]]]


In [204]:
for i, fn in enumerate(f):
    print(fn)
    s3 = calculate.Sort.fragmentSort(struct[i])
    frag2d, fragStruct3d = [], []
    for s2 in s3:   
        frag, fragStruct = calculate.Convert.fragmentConvert(s2)
        frag2d.append(frag)
        fragStruct3d.append(fragStruct)
    frag2d, fragStruct3d = calculate.Sort.dicectionSort(frag2d, fragStruct3d)
    print(frag2d)
    print(fragStruct3d)
    model_ind = SGmodel.induction(frag2d, fragStruct3d, basises[i])
    print('-----------------result---------------')
    predExc = model_ind.run(Rphi=3, Rrange=1)
    print(predExc)
    print(allEnergy[i][2])
    if molType[i] == 'Dimer':
        name = fn[:-8] + '-dimer'
    else:
        name = fn[:-8]
    resultData.saveResult(name, 'ind', basises[i], refEnergy=allEnergy[i][2], predEnergy=predExc)

formaldehyde-formamide_jtz.out
[['H'], ['F']]
[[[-1.2853507142857143, -0.13495352380952383, -6.809523809523811e-05]], [[1.9451395714285713, 0.14486342857142856, 8.857142857142859e-05]]]
-----------------result---------------
Ind_component:  ['FH'] 
 Ind_component_quantity:  [1] 
 Ind_component_distance:  [[3.2425861650044974]] 

-3.9754610599948386
-3.97546106
formaldehyde-acetamide_jtz.out
[['A', 'H'], ['F']]
[[[-2.666553, -0.030556, -0.0002], [-0.7152634761904761, -0.11401357142857152, 0.00017666666666666666]], [[2.520709714285714, 0.1344018571428571, -0.00016714285714285716]]]
-----------------result---------------
Ind_component:  ['FH'] 
 Ind_component_quantity:  [1] 
 Ind_component_distance:  [[3.2454942355268437]] 

-3.9571254067711856
-3.92858612
propanal-formamide_jtz.out
[['H'], ['F', 'B', 'A']]
[[[-2.4646841428571427, 0.180914, -3.666666666666666e-05]], [[0.7339751428571429, -0.4169732857142857, 0.00010285714285714286], [2.302742, 0.775897, 2e-05], [3.357973, -0.315014, -0.00

In [198]:
inddd=0
print(f[inddd])
struct3D = struct[inddd]
print(struct3D)
basis = basises[inddd]
moltype = molType[inddd]
fileName = f[inddd]
print(basis)
referenceEnergy = allEnergy[inddd]
s3 = calculate.Sort.fragmentSort(struct3D)
# print(s3[0])
frag2d, fragStruct3d = [], []
for s2 in s3:   
    frag, fragStruct = calculate.Convert.fragmentConvert(s2)
    frag2d.append(frag)
    fragStruct3d.append(fragStruct)

frag2d, fragStruct3d = calculate.Sort.dicectionSort(frag2d, fragStruct3d)
frag2d, fragStruct3d

ethene-formamide_jtz.out
[[['C', '-1.766373', '-0.098848', '0.00001'], ['O', '-1.193645', '-1.176729', '-0.00013'], ['N', '-1.144688', '1.105123', '0.00005'], ['H', '-1.676810', '1.955452', '0.00019'], ['H', '-0.136231', '1.139725', '-0.00000'], ['H', '-2.863016', '-0.012562', '0.00008']], [['C', '2.052378', '-0.048776', '-0.66772'], ['C', '2.052280', '-0.048966', '0.66776'], ['H', '2.803540', '0.487741', '-1.23133'], ['H', '1.300802', '-0.599948', '-1.21688'], ['H', '1.300620', '-0.600291', '1.21666'], ['H', '2.803360', '0.487392', '1.23163']]]
jtz


([['H'], ['C']],
 [[[-1.3409625714285716, -0.10814566666666658, -2.9999999999999997e-05]],
  [[2.0523290000000003, -0.048871, 2.000000000005701e-05]]])

In [199]:
model_ind = SGmodel.induction(frag2d, fragStruct3d, basis)
model_ind.run(Rphi=4, Rrange=1, train_mode=True, referenceInd=referenceEnergy[2])

Ind_component:  ['CH'] 
 Ind_component_quantity:  [1] 
 Ind_component_distance:  [[3.393809242921091]] 

['CH', -1.1839944657578831, 4]


array([['AA', '-0.033715494', '4'],
       ['AB', '-0.036913772', '4'],
       ['BB', '-0.02836732', '4'],
       ['CC', '-0.193321969', '4'],
       ['AC', '-0.070760626', '4'],
       ['BC', '-0.133720625', '4'],
       ['DD', '-0.40372383', '4'],
       ['AD', '-0.229345916', '4'],
       ['BD', '0.003247029', '4'],
       ['EE', '-1.967417718', '4'],
       ['AE', '-0.124705575', '4'],
       ['BE', '-0.424240364', '4'],
       ['CE', '-0.989771304', '4'],
       ['EG', '-5.453622319', '4'],
       ['EH', '-3.893678911', '4'],
       ['FF', '-2.128561565', '3'],
       ['AF', '-0.486394944', '3'],
       ['BF', '-0.848970053', '3'],
       ['CF', '-0.785597518', '3'],
       ['FG', '-7.922329595', '3'],
       ['FH', '-5.392143955', '3'],
       ['GG', '-16.36323396', '3'],
       ['AG', '-0.722578873', '4'],
       ['BG', '-0.089556675', '4'],
       ['CG', '-2.038692011', '4'],
       ['GH', '-14.91397635', '3'],
       ['HH', '-12.81114542', '3'],
       ['AH', '-0.53488856', '4